# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [15]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_target = pd.DataFrame(boston.target, columns = ["MEDV"])
boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)


# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [16]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
#boston_features.head()
boston_features.sample(5)

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
268,0.465723,-1.006707,0.0,7.470,-0.568077,0.393147,-2.750363,0.983358,-2.032981,1,0,1,0,0
411,0.806782,0.947825,0.0,6.657,1.117494,0.127139,0.784047,0.087574,1.139400,0,1,0,0,1
166,0.603227,1.049081,0.0,7.929,0.982364,0.250249,-1.764795,0.930405,-1.770177,1,0,0,0,1
269,0.278808,-0.283441,1.0,5.920,-0.251588,0.523941,0.122279,0.985980,0.404424,1,0,1,0,0
187,0.264076,-1.191313,0.0,6.782,-0.977023,0.509845,-1.496563,0.992360,-0.786023,1,0,0,0,1


In [4]:
boston_features.dtypes

CRIM              float64
INDUS             float64
CHAS              float64
RM                float64
AGE               float64
DIS               float64
PTRATIO           float64
B                 float64
LSTAT             float64
RAD_(0, 6]          uint8
RAD_(6, 24]         uint8
TAX_(0, 270]        uint8
TAX_(270, 360]      uint8
TAX_(360, 712]      uint8
dtype: object

In [5]:
boston_features.isnull().sum()

CRIM              0
INDUS             0
CHAS              0
RM                0
AGE               0
DIS               0
PTRATIO           0
B                 0
LSTAT             0
RAD_(0, 6]        0
RAD_(6, 24]       0
TAX_(0, 270]      0
TAX_(270, 360]    0
TAX_(360, 712]    0
dtype: int64

In [6]:
#looking at mean and # standard deviations from mean
boston_features.describe()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
count,506.000000,5.060000e+02,506.000000,506.000000,5.060000e+02,506.000000,5.060000e+02,506.000000,5.060000e+02,506.000000,506.000000,506.000000,506.000000,506.000000
mean,0.448432,3.510587e-16,0.069170,6.284634,-1.474446e-16,0.449191,-2.878681e-15,0.898568,2.808469e-16,0.658103,0.341897,0.195652,0.343874,0.460474
std,0.226336,1.000990e+00,0.253994,0.702617,1.000990e+00,0.227318,1.000990e+00,0.230205,1.000990e+00,0.474815,0.474815,0.397094,0.475470,0.498929
min,0.000000,-3.783365e+00,0.000000,3.561000,-2.335437e+00,0.000000,-3.000989e+00,0.000000,-3.036568e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.268367,-6.614858e-01,0.000000,5.885500,-8.374480e-01,0.261281,-4.125447e-01,0.945730,-7.200364e-01,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.387692,1.428756e-01,0.000000,6.208500,3.173816e-01,0.439687,3.139590e-01,0.986232,9.850402e-02,1.000000,0.000000,0.000000,0.000000,0.000000
75%,0.666445,9.478254e-01,0.000000,6.623500,9.067981e-01,0.642307,7.840469e-01,0.998298,7.656165e-01,1.000000,1.000000,0.000000,1.000000,1.000000
max,1.000000,1.497881e+00,1.000000,8.780000,1.117494e+00,1.000000,1.468580e+00,1.000000,2.108674e+00,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
#look for variables - with pd.concat[boston_features, boston_target], axis =1).corr()['price']
#check string correlation and where its positivie
#correlation of price (it is negagive)

In [17]:
boston2 = pd.concat([boston_features, boston_target], axis =1)

In [8]:
boston2.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]",MEDV
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0,24.0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0,21.6
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0,34.7
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0,33.4
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0,36.2


In [9]:
boston2.corr()['MEDV']

CRIM             -0.454302
INDUS            -0.519270
CHAS              0.175260
RM                0.695360
AGE              -0.376955
DIS               0.292316
PTRATIO          -0.503160
B                 0.333461
LSTAT            -0.815442
RAD_(0, 6]        0.246441
RAD_(6, 24]      -0.246441
TAX_(0, 270]      0.373825
TAX_(270, 360]    0.091468
TAX_(360, 712]   -0.384692
MEDV              1.000000
Name: MEDV, dtype: float64

## Run an linear model in Statsmodels

In [10]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

formula = "MEDV ~ LSTAT"
model = ols(formula= formula, data=(boston2)).fit()
#model.summary()

In [11]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.664
Method:                 Least Squares   F-statistic:                     1000.
Date:                Sun, 12 May 2019   Prob (F-statistic):          9.28e-122
Time:                        10:39:40   Log-Likelihood:                -1563.6
No. Observations:                 506   AIC:                             3131.
Df Residuals:                     504   BIC:                             3140.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     22.5328      0.237     95.116      0.000      22.067      22.998
LSTAT         -7.4923      0.237    -31.627      0.000      -7.958      -7.027
==============================================================================
Omnibus:                      126.181   Durbin-Watson:                   0.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              323.855
Skew:                           1.237   Prob(JB):                     4.74e-71
Kurtosis:                       6.039   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
#import statsmodels.api as sm
#X_int = sm.add_constant(X)
#model = sm.OLS(y,X_int).fit()
#model.summary(

In [19]:
formula = "MEDV ~ LSTAT+RM+INDUS+CRIM+PTRATIO"
model = ols(formula= formula, data=boston2).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.733
Method:                 Least Squares   F-statistic:                     278.6
Date:                Sun, 12 May 2019   Prob (F-statistic):          5.32e-142
Time:                        10:48:45   Log-Likelihood:                -1503.4
No. Observations:                 506   AIC:                             3019.
Df Residuals:                     500   BIC:                             3044.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.1234      2.562      0.829      0.408      -2.911       7.158
LSTAT         -5.6166      0.344    -16.348      0.000      -6.292      -4.942
RM             3.1170      0.415      7.506      0.000       2.301       3.933
INDUS          0.0692      0.340      0.203      0.839      -0.599       0.737
CRIM           1.8287      1.469      1.245      0.214      -1.058       4.716
PTRATIO       -1.7563      0.240     -7.322      0.000      -2.228      -1.285
==============================================================================
Omnibus:                      161.210   Durbin-Watson:                   0.942
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              714.940
Skew:                           1.357   Prob(JB):                    5.66e-156
Kurtosis:                       8.152   Cond. No.                         79.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
outcome = 'MEDV'
predictors = boston2.drop('MEDV', axis=1)
pred_sum = "+".join(predictors.columns)
formula = outcome + "~" + pred_sum
model = ols(formula= formula, data=(boston2)).fit()
model.summary()


SyntaxError: invalid syntax (<unknown>, line 1)

## Run the same model in Scikit-learn

In [ ]:
from sklearn.linear_model import LinearRegression
predictors = boston2.drop(LSTAT+RM+INDUS+CRIM+PTRATIO, axis=1)
y = boston2['MEDV']
linreg = LinearRegression()
linreg.fit(predictors, y)

linreg.coef_

linreg.intercept_

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

### Statsmodels

### Scikit-learn

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

In [ ]:
#PTratio negative value - nagative ration ptratio and price


## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

In [ ]:
#have to transform eeach - take code from top of lab, ran again,a d didd transformation minmax scaling, logcrim -

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.